# Finetunning LLaMa 7B for sentiment analysis

1. Install dependencies
2. Load model
3. Test prompt
4. Create insturction tuned dataset
5. Run training
6. Test prompt

In [2]:
!pip install transformers accelerate datasets trl

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
!pip install torch

In [ ]:
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-2-7b-hf"
access_token = userdata.get('hf_access_token')

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16, use_auth_token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=access_token)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following setiment into a number.

### Input:
It was a great movie!

### Response:"""

model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
with torch.cuda.amp.autocast():
  output = model.generate(**model_inputs, max_new_tokens=50)

In [ ]:
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

# Fine Tune with HuggingFace IMDB data

In [ ]:
from datasets import load_dataset
train_ds, test_ds = load_dataset("imdb", split=["train[:2%]+train[-2%:]", "test[:2%]+test[-2%:]"])

sentiment_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following setiment into a number.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts(examples):
  inputs = examples["text"]
  outputs = examples["label"]
  texts = []
  for input, output in zip(inputs, outputs):
    text = sentiment_prompt.format(input, output) + EOS_TOKEN
    texts.append(text)
  return {"texts": texts,}
pass

trains_ds = train_ds.shuffle(seed=42).map(formatting_prompts, batched=True)
tests_ds = test_ds.shuffle(seed=42).map(formatting_prompts, batched=True)

In [ ]:
print(train_ds)
print(test_ds)
print(trains_ds["text"][5])
print(trains_ds["label"][:10])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    fp16= not torch.cuda.is_bf16_supported(),
    bf16= torch.cuda.is_bf16_supported(),
    logging_steps=10,

)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=trains_ds,
    eval_dataset=tests_ds,
    dataset_text_field="text",
    max_seq_length=256,
    args=args
)


In [ ]:
import gc
def report_gpu():
   print(torch.cuda.list_gpu_processes())
   gc.collect()
   torch.cuda.empty_cache()

report_gpu()
trainer.train()